In [1]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
from census import Census

In [2]:
from config import (G_key, Census_key)
gmaps.configure(api_key=G_key)

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [3]:
Vacation_df = pd.read_csv('Clean_city_data.csv')
Vacation_df.dropna(inplace = True)
Vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pevek,69.70,170.31,3.42,84,18,1.36,RU,1594770124
1,Mayo,38.89,-76.51,29.44,51,1,2.60,US,1594770124
2,Tuktoyaktuk,69.45,-133.04,14.00,76,90,4.10,CA,1594770124
3,Tazmalt,36.39,4.40,25.00,78,40,3.10,DZ,1594770124
4,Mataura,-46.19,168.86,8.30,80,47,1.81,NZ,1594770124


Humidity Heatmap

In [4]:
locations = Vacation_df[["Lat", "Lng"]]
Humidity = Vacation_df["Humidity"]

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

In [6]:
Prefered_Max_Temp = (Vacation_df["Max Temp"] < 30)&(Vacation_df["Max Temp"] > 20)
Prefered_Cloudiness = Vacation_df["Cloudiness"] < 5
Prefered_Wind_Speed = Vacation_df["Wind Speed"] < 5
Prefered_Weather = Vacation_df[Prefered_Max_Temp&Prefered_Cloudiness&Prefered_Wind_Speed]
Prefered_Weather = Prefered_Weather.dropna()
Prefered_Weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Mayo,38.89,-76.51,29.44,51,1,2.60,US,1594770124
9,Albany,42.60,-73.97,26.67,57,2,1.93,US,1594770125
15,Berdigestyakh,62.10,126.70,24.88,39,0,1.99,RU,1594770125
36,Aswān,24.09,32.91,29.27,14,3,4.36,EG,1594770128
72,Jamestown,42.10,-79.24,22.78,60,1,4.10,US,1594770131


Hotel Map

In [12]:
hotel_df = Prefered_Weather
hotel_df["Hotel Name"] = ""

In [13]:
params = {"radius": 5000,"type": "lodging","keyword":"hotel","key": G_key}

In [14]:
params["location"] = f"{Lat},{Lng}"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [15]:
for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    params["location"] = f"{Lat},{Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
   
    try:
        print(f"{response['results'][0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Sofitel Legend Old Cataract Aswan.
------------
Chautauqua Harbor Hotel.
------------
Gostevoy Dom S.barguzin "Vizit".
------------
Azul Singular.
------------
Hotel hewa bora.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Mauricia Beachcomber Resort & Spa.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Mar Doce de Abrolhos.
------------
فندق جراند بالاس Grand Palace Hotel.
------------
SunPrime C-lounge Hotel.
------------
Graciosa Resort - Biosphere Island Hotel.
------------
Hattusa Vacation Thermal Club Erzin.
------------
The Sign Sile Hotel & Spa.
------------
Missing field/result... skipping.
------------
Desert Gat.
------------
Holiday Inn Nola - Naples Vulcano Buono.
------------
Therasia Resort.
------------
Hotel Shi

In [16]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Mayo,38.89,-76.51,29.44,51,1,2.60,US,1594770124,
9,Albany,42.60,-73.97,26.67,57,2,1.93,US,1594770125,
15,Berdigestyakh,62.10,126.70,24.88,39,0,1.99,RU,1594770125,
36,Aswān,24.09,32.91,29.27,14,3,4.36,EG,1594770128,Sofitel Legend Old Cataract Aswan
72,Jamestown,42.10,-79.24,22.78,60,1,4.10,US,1594770131,Chautauqua Harbor Hotel
90,Barguzin,53.62,109.62,23.56,59,0,2.38,RU,1594770134,"Gostevoy Dom S.barguzin ""Vizit"""
110,Ribeira Grande,38.52,-28.70,21.06,78,0,3.20,PT,1594770136,Azul Singular
113,Manono,-7.30,27.42,20.14,48,3,1.43,CD,1594770136,Hotel hewa bora
115,Sidi Ali,36.10,0.42,21.96,78,2,3.10,DZ,1594770136,
124,Smidovich,48.60,133.82,23.17,83,3,1.98,RU,1594770137,


In [17]:
coordinates = hotel_df[["Lat","Lng"]].astype(float)
hotel_df = hotel_df.dropna()
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [18]:
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [19]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [20]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [22]:
hotel_layer = gmaps.symbol_layer(
    coordinates, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))